## Getting started

### Set up environment

In [ ]:
import numpy as np
import pandas as pd
import random
import torch
import transformers
import warnings

from IPython.display import display, HTML
from typing import Any, Dict, List, Optional

transformers.set_seed(42)
warnings.simplefilter("ignore")

### Initialize Ray runtime

In [ ]:
import ray

In [ ]:
ray.init()

### Open the Ray Dashboard

## Data ingest

### Load the dataset

In [ ]:
from datasets import load_dataset

In [ ]:
hf_dataset = load_dataset("tatsu-lab/alpaca", split="train").train_test_split(
    test_size=0.2
)
hf_dataset

### Display sample data

In [ ]:
def show_random_elements(dataset: List, num_examples: int = 2) -> None:
    """
    Picks a random subset of elements from the given dataset and displays them
    as a Pandas DataFrame.

    Args:
        dataset: A list of elements to choose from.
        num_examples: The number of elements to choose. Defaults to 2.

    Raises:
        ValueError: If `num_examples` is greater than the length of `dataset`.

    Returns:
        None
    """

    if num_examples > len(dataset):
        raise ValueError("Can't pick more elements than there are in the dataset.")

    picks = random.sample(range(len(dataset)), k=num_examples)
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


show_random_elements(hf_dataset["train"])

### Convert to Ray Dataset

In [ ]:
ray_dataset = ray.data.from_huggingface(hf_dataset)
ray_dataset

## Distributed preprocessing

### Implement preprocessing function

In [ ]:
from ray.data.preprocessors import BatchMapper
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
def preprocess_function(batch: Dict[str, Any]) -> Dict[str, Any]:
    """
    Tokenizes the input and instruction pairs in a batch using the T5 tokenizer
    from the Google/Flan-T5-Base model, and returns a dictionary containing the
    encoded inputs and labels.

    Args:
        batch: A dictionary containing at least two keys, "instruction" and 
        "input", whose values are lists of strings.

    Returns:
        A dictionary containing the encoded inputs and labels, as returned by 
        the T5 tokenizer.
    """
    model_name = "google/flan-t5-base"
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    encoded_inputs = tokenizer(
        list(batch["instruction"]),
        list(batch["input"]),
        padding=True,
        truncation=True,
        return_tensors="np",
    )

    encoded_inputs["labels"] = encoded_inputs["input_ids"].copy()

    return dict(encoded_inputs)

In [ ]:
batch_preprocessor = BatchMapper(preprocess_function, batch_format="pandas")

## Distributed finetuning

### Initialize training logic for each worker

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
batch_size = 2
use_gpu = True


def trainer_init_per_worker(
    train_dataset: ray.data.Dataset,
    eval_dataset: Optional[ray.data.Dataset] = None,
    **config,
) -> Trainer:
    """
    Initializes a Hugging Face Trainer for training a T5 text generation model.

    Args:
        train_dataset (ray.data.Dataset): The dataset for training the model.
        eval_dataset (ray.data.Dataset, optional): The dataset for evaluating 
        the model.
            Defaults to None.
        **config: Additional arguments to configure the Trainer.

    Returns:
        Trainer: A Hugging Face Trainer for training the T5 model.
    """
    device = torch.device("cuda" if use_gpu and torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model_name = "google/flan-t5-base"

    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    training_args = TrainingArguments(
        "flan-t5-base-finetuned-alpaca",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=config.get("learning_rate", 2e-5),
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=config.get("epochs", 2),
        weight_decay=config.get("weight_decay", 0.01),
        push_to_hub=False,
        disable_tqdm=True,
        no_cuda=not use_gpu,
    )

    hf_trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

    print("Starting training")
    return hf_trainer

### Define Trainer

In [ ]:
from ray.train.huggingface import HuggingFaceTrainer
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig

In [ ]:
SMALL_DATA = True

if SMALL_DATA:
    train_dataset = ray_dataset["train"].limit(5)
    validation_dataset = ray_dataset["test"].limit(5)
else:
    train_dataset = ray_dataset["train"]
    validation_dataset = ray_dataset["test"]

In [ ]:
num_workers = 2

trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
    datasets={
        "train": train_dataset,
        "evaluation": validation_dataset,
    },
    run_config=RunConfig(
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="eval_loss",
            checkpoint_score_order="min",
        ),
    ),
    preprocessor=batch_preprocessor,
)

### Run finetuning

In [ ]:
result = trainer.fit()

### Try the finetuned model

In [ ]:
model_name = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
checkpoint = result.checkpoint
finetuned_model = checkpoint.get_model(model)

In [ ]:
instruction = "How many bees do I have?"  # Enter your own instruction here.
input = "I don't have enough bees."  # Write additional context for the model here.

inputs = tokenizer(instruction, input, return_tensors="pt")
outputs = finetuned_model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

## [Optional] Distributed hyperparameter tuning

In [ ]:
from ray import tune
from ray.tune import Tuner
from ray.tune.schedulers.async_hyperband import ASHAScheduler

total_num_trials = 4
max_tune_epochs = 16

tuner = Tuner(
    trainer,
    param_space={
        "trainer_init_config": {
            "learning_rate": tune.choice([2e-5, 2e-4, 2e-3, 2e-2]),
            "epochs": tune.choice([2, 4, 8, max_tune_epochs]),
            "weight_decay": tune.choice([0.01, 0.1, 1.0, 10.0]),
        }
    },
    tune_config=tune.TuneConfig(
        metric="eval_loss",
        mode="min",
        num_samples=total_num_trials,
        scheduler=ASHAScheduler(
            max_t=max_tune_epochs,
        ),
    ),
    run_config=RunConfig(
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="eval_loss",
            checkpoint_score_order="min",
        )
    ),
)

In [ ]:
result_grid = tuner.fit()

## Distributed batch inference

In [ ]:
class HuggingFaceModelPredictor(Predictor):
    """
    A Ray Predictor for HuggingFace models that generates text given input data.

    Args:
        model (transformers.PreTrainedModel): A trained HuggingFace model.
        tokenizer (Optional[transformers.PreTrainedTokenizerBase]): A tokenizer 
        that can tokenize input text.
        preprocessor (Optional[Callable]): A function that takes raw input data 
        and returns tokenized input data.
        use_gpu (bool): Whether to use a GPU or CPU for prediction.
    """

    def __init__(
        self,
        model: Any,
        tokenizer: Optional[Any] = None,
        preprocessor: Optional[Any] = None,
        use_gpu: bool = False,
    ) -> None:
        super().__init__(preprocessor)
        self.model = model
        self.use_gpu = use_gpu
        self.tokenizer = tokenizer

    @classmethod
    def from_checkpoint(
        cls,
        checkpoint: Any,
        model_cls: Any,
        *,
        tokenizer: Optional[Any] = None,
        use_gpu: bool = False,
        **get_model_kwargs: Any,
    ) -> "HuggingFaceModelPredictor":
        """
        Create a HuggingFaceModelPredictor from a checkpoint.

        Args:
            checkpoint (Any): A checkpoint containing a trained HuggingFace model.
            model_cls (Any): The type of HuggingFace model to load from the checkpoint.
            tokenizer (Optional[Any]): A tokenizer that can tokenize input text.
            use_gpu (bool): Whether to use a GPU or CPU for prediction.
            **get_model_kwargs (Any): Additional keyword arguments for loading 
            the HuggingFace model.

        Returns:
            HuggingFaceModelPredictor: A Ray Predictor for the HuggingFace model.
        """
        if not tokenizer:
            tokenizer = AutoTokenizer
        if isinstance(tokenizer, type):
            tokenizer = checkpoint.get_tokenizer(tokenizer)
        return cls(
            checkpoint.get_model(model_cls, **get_model_kwargs),
            tokenizer=tokenizer,
            preprocessor=checkpoint.get_preprocessor(),
            use_gpu=use_gpu,
        )

    def _predict_numpy(
        self,
        data: Dict[str, Any],
        feature_columns: Optional[List[str]] = None,
        **generate_kwargs: Any,
    ) -> pd.DataFrame:
        """
        Generates text given input data.

        Args:
            data (Dict[str, Any]): A dictionary of input data.
            feature_columns (Optional[List[str]]): A list of feature column names 
            to use for prediction.
            **generate_kwargs (Any): Additional keyword arguments for generating text.

        Returns:
            pd.DataFrame: A Pandas DataFrame with a single column "generated_output"
            containing the generated text.
        """
        # we get already tokenized text here because we have the tokenizer as an AIR preprocessor
        if feature_columns:
            data = {k: v for k, v in data.items() if k in feature_columns}

        data = {
            k: torch.from_numpy(v).to(device=self.model.device) for k, v in data.items()
        }
        generate_kwargs = {**data, **generate_kwargs}

        outputs = self.model.generate(**generate_kwargs)
        return pd.DataFrame(
            self.tokenizer.batch_decode(outputs, skip_special_tokens=True),
            columns=["generated_output"],
        )


predictor = BatchPredictor.from_checkpoint(
    checkpoint=result.checkpoint,
    predictor_cls=HuggingFaceModelPredictor,
    model_cls=T5ForConditionalGeneration,
    tokenizer=T5Tokenizer,
    use_gpu=use_gpu,
    device_map="auto",
    torch_dtype=torch.float16,
)

In [ ]:
prediction = predictor.predict(
    validation_dataset,
    num_gpus_per_worker=int(use_gpu),
    batch_size=256,
    max_new_tokens=128,
)

In [ ]:
prediction.to_pandas()